# **机器翻译**

## **Seq2Seq**

Seq2Seq模型适用于输入和输出都不是定长序列的情况，Seq2Seq模型由编码器(encoder)和解码器(decoder)组成

这里引入了两个特殊的符号“&lt;bos&gt;” 和 “&lt;eos&gt;” 。在训练数据集中，我们可以在每个句子后面附上符号 “&lt;eos&gt;”表示句子的结束，因此编码器每个时间步的输入就是
句子的内容，标点和 “&lt;eos&gt;”标记。编码器用于提取句子编码信息并将句子信息提供给解码器，解码器依据句子编码正确输出翻译句子的内容包括句子的结束 “&lt;eos&gt;” ，
而解码器的初始时间步输入是表示句子开始的符号 “&lt;bos&gt;”

### **编码器**

编码器的作用是将不定长的输入序列变成一个定长的背景向量$c$，并在该背景中编码输入序列信息。常用的编码器是循环神经网络或transfomer

我们把单个神经元的作用写在这里

$$
h_t = f(x_t, h_{t-1}) \hspace{3em} \text{非LSTM} \\
h_t = f(x_t, h_{t-1}, c_{t-1}) \hspace{3em}  \text{LSTM}
$$

那么编码器的作用就是：

$$c = q(h_1,h_2,.....,h_T)$$

这里的$q$我们可以选择是最后一个隐层也可以是平均值

### **解码器**

解码器在时间步t的输出依赖于之前的输出以及编码器的输入，解码器得到的结果其实是

$$P(y_{t^*}|y_1, .....,y_{t^*-1},c)$$

为此，其实解码器当前时间步的隐层状态的计算如下所示

$$s^* = g(y_{t^*-1}, c, s_{t^*-1})$$

### **训练**

由最大似然估计，我们需要最大化的概率是

$$
\begin{aligned}
P(y_1, \ldots, y_{T'} \mid x_1, \ldots, x_T)
&= \prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, x_1, \ldots, x_T)\\
&= \prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c}),
\end{aligned}
$$

对数极大似然

$$-\log P(y_1, \ldots, y_{T'} \mid x_1, \ldots, x_T) = -\sum_{t'=1}^{T'} \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c})$$

## **Beam Search**

那么如何使用Seq2Seq来预测具体的每个句子呢？我们假设输出文本的词典为$\mathcal Y$,输出序列的最大长度为$T'$,那么最后的输出序列总共有
$|\mathcal Y|^{T'}$种

### **贪婪搜索**

对于每一个时间步$t'$,我们从所有词种找到概率最大的词

$$
y _ { t ^ { \prime } } = \underset { y \in \mathcal { Y } } { \operatorname { argmax } } P \left( y | y _ { 1 } , \ldots , y _ { t ^ { \prime } - 1 } , c \right)
$$

一旦输出了&lt;eos&gt;我们就认为结束了搜索

<div align=center>
<img width="200" src="../image/10.10_s2s_prob1.svg"/>
</div>
<div align=center>图10.9 在每个时间步，贪婪搜索选取条件概率最大的词</div>

以上图为例，当时间步2选择了B后，时间步3的条件最大概率是0.4，最终句子的概率为$0.5 \times 0.4 \times 0.4 \times 0.6 = 0.048$

<div align=center>
<img width="200" src="../image/10.10_s2s_prob2.svg"/>
</div>
<div align=center>图10.10 在时间步2选取条件概率第二大的词“C”</div>

再如上图,当第二步选择了C后，后续的概率如图，句子概率为$0.5 \times 0.3 \times 0.6 \times 0.6 = 0.054$

因此，贪婪搜索不是最佳的句子

### **beam Search**

束搜索（beam search）是对贪婪搜索的一个改进算法。它有一个束宽（beam size）超参数。我们将它设为$k$。在时间步1时，选取当前时间步条件概率最大的$k$个词，分别组成$k$个候选输出序列的首词。在之后的每个时间步，基于上个时间步的$k$个候选输出序列，从$k\left|\mathcal{Y}\right|$个可能的输出序列中选取条件概率最大的$k$个，作为该时间步的候选输出序列。最终，我们从各个时间步的候选输出序列中筛选出包含特殊符号“&lt;eos&gt;”的序列，并将它们中所有特殊符号“&lt;eos&gt;”后面的子序列舍弃，得到最终候选输出序列的集合。

<div align=center>
<img width="500" src="../image/10.10_beam_search.svg"/>
</div>
<div align=center>图10.11 束搜索的过程。束宽为2，输出序列最大长度为3。候选输出序列有A、C、AB、CE、ABD和CED</div>

在最终候选输出序列的集合中，我们取以下分数最高的序列作为输出序列：

$$ \frac{1}{L^\alpha} \log P(y_1, \ldots, y_{L}) = \frac{1}{L^\alpha} \sum_{t'=1}^L \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \boldsymbol{c}),$$

其中$L$为最终候选序列长度，$\alpha$一般可选为0.75。分母上的$L^\alpha$是为了惩罚较长序列在以上分数中较多的对数相加项。分析可知，束搜索的计算开销为$\mathcal{O}(k\left|\mathcal{Y}\right|T')$。这介于贪婪搜索和穷举搜索的计算开销之间。此外，贪婪搜索可看作是束宽为1的束搜索。束搜索通过灵活的束宽$k$来权衡计算开销和搜索质量。

## **attention机制**

解码器不同位置词语的输出其实需要用到的信息是不一样的，原句子的词语对本时间步的输出影响有所不同。让我们来看下属公式：

$$s^* = g(y_{t^*-1}, c, s_{t^*-1})$$

每个时间步的c可以不同，从解码器的输出不同变化得到，那么这个公式就变成了：

$$s' = g(y_{t'-1}, c_{t'-1}, s_{t'-1})$$

### **背景变量的计算**

我们把$c_{t'-1}$称为背景变量，那么如何计算背景变量呢?

<div align=center>
<img width="500" src="../image/10.11_attention.svg"/>
</div>
<div align=center>图10.12 编码器—解码器上的注意力机制</div>

给编码器的隐藏状态的每个输出一个权重，那么

$$c_{t'} = \sum_{t=1}^{T}a_{t't}{h_t}$$

那么问题就转换成为了权重$a_{t't}$的计算

$$a_{t't} = \frac {exp(e_{t't})}{\sum_{k=1}^Texp(e_{t'k})}$$

我们使用softmax的方式计算权重，那么$e_{t't}$怎么计算

$$e_{t't} = a(s_{t'-1}, h_t)$$

这里的计算有很多种方式，最简单的计算方式是计算$s_{t'-1}$和$h_t$的内积，当然还有别的方式，最早的方式设定了三个可训练张量

$$a(\boldsymbol{s}, \boldsymbol{h}) = \boldsymbol{v}^\top \tanh(\boldsymbol{W}_s \boldsymbol{s} + \boldsymbol{W}_h \boldsymbol{h}),$$

### **矢量化计算**

我们假设$s_{t'-1}$为$Q \in \mathbb R^{(1 \times h)}$, 令编码器的输出矩阵$H$为$H = K = V \in \mathbb R^{(T \times h)}$

那么矢量化计算为

$$softmax(QK^T)V$$

以此得到的解码器计算为(以GPU为例)

$$
\begin{aligned}
\boldsymbol{r}_{t'} &= \sigma(\boldsymbol{W}_{yr} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{sr} \boldsymbol{s}_{t' - 1} + \boldsymbol{W}_{cr} \boldsymbol{c}_{t'} + \boldsymbol{b}_r),\\
\boldsymbol{z}_{t'} &= \sigma(\boldsymbol{W}_{yz} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{sz} \boldsymbol{s}_{t' - 1} + \boldsymbol{W}_{cz} \boldsymbol{c}_{t'} + \boldsymbol{b}_z),\\
\tilde{\boldsymbol{s}}_{t'} &= \text{tanh}(\boldsymbol{W}_{ys} \boldsymbol{y}_{t'-1} + \boldsymbol{W}_{ss} (\boldsymbol{s}_{t' - 1} \odot \boldsymbol{r}_{t'}) + \boldsymbol{W}_{cs} \boldsymbol{c}_{t'} + \boldsymbol{b}_s),
\end{aligned}
$$

$$\boldsymbol{s}_{t'} = \boldsymbol{z}_{t'} \odot \boldsymbol{s}_{t'-1}  + (1 - \boldsymbol{z}_{t'}) \odot \tilde{\boldsymbol{s}}_{t'},$$

# **代码实现**

In [7]:
import collections
import os, io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data

import sys
sys.path.append('../utils')
import d2lzh as d2l

In [8]:
PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'

In [9]:
device = torch.device('cuda')

## **读取和预处理数据**

In [10]:
# 将一个序列中所有的词记录在all_tokens中以便之后构造词典，然后在该序列后面添加PAD直到序列
# 长度变为max_seq_len，然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    all_tokens.extend(seq_tokens)
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1)
    all_seqs.append(seq_tokens)
    
def build_data(all_tokens, all_seqs):
    vocab = Vocab.Vocab(collections.Counter(all_tokens), specials=[PAD, BOS, EOS])
    indices = [[vocab.stoi[w] for w in seq] for seq in all_seqs] # 将词转变为indices
    return vocab, torch.tensor(indices)

我们使用一个很小的法语转英语的数据集。我们需要为法语和英语分别创建词典，二者相互独立

In [11]:
def read_data(max_seq_len):
    # 在这里我们把两种语言的max_seq_len设置为一样
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []
    with io.open('../datasets/fr-en-small.txt') as f:
        lines = f.readlines()
    for line in lines:
        in_seq, out_seq = line.rstrip().split('\t')
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:
            continue # 如果加上eos后长度比max_seq_len要大那么就要放弃本条数据
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)
    in_vocab, in_data = build_data(in_tokens, in_seqs)
    out_vocab, out_data = build_data(out_tokens, out_seqs)
    return in_vocab, out_vocab, Data.TensorDataset(in_data, out_data)

In [12]:
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len)
dataset[0]

(tensor([ 5,  4, 45,  3,  2,  0,  0]), tensor([ 8,  4, 27,  3,  2,  0,  0]))

### **含注意力机制的编码器和解码器**

#### **编码器**

我们以GRU为例，pytorch中的GRU本身会输出最终时间步的多层隐藏状态

In [21]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)
        
    def forward(self, inputs, state):
        embedding = self.embedding(inputs.long()).permute(1, 0, 2)
        return self.rnn(embedding, state)
    
    def begin_state(self):
        return None # 不输入state时，自动初始化为0

In [22]:
encoder = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
output, state = encoder(torch.zeros((4, 7)), encoder.begin_state())
output.shape, state.shape # GRU的state是h, 而LSTM的是一个元组(h, c)

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

#### **attention_model**

我们使用较为传统的attention方式

将输入连接后通过含单隐层的多层感知机交换。其中隐藏层数输入事解码器单时间上的隐藏状态和编码器在时间步上隐藏状态的**一一连接**, 输出为1，同时向量$V$的长度是一个超参数attention_size

$$a(\boldsymbol{s}, \boldsymbol{h}) = \boldsymbol{v}^\top \tanh(\boldsymbol{W}_s \boldsymbol{s} + \boldsymbol{W}_h \boldsymbol{h}),$$

In [25]:
def attention_model(input_size, attention_size):
    # (m, seq_len, input_size) -> (m, seq_len, 1)
    model = nn.Sequential(nn.Linear(input_size, attention_size, bias=False),
                          nn.Tanh(),
                          nn.Linear(attention_size, 1, bias=False))
    return model

编码器和解码器的隐藏单元数要相同

In [28]:
def attention_forward(model, enc_states, dec_state):
    """
    enc_states:(time_step, batch_size, hidden_states)
    dec_state:(batch_size, hidden_state)
    """
    # 扩展dec_state
    dec_states = dec_state.unsqueeze(0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)
    alpha = F.softmax(e, dim=0)
    return (alpha * enc_states).sum(dim=0)

In [29]:
seq_len, batch_size, num_hiddens = 10, 4, 8
model = attention_model(2*num_hiddens, 10) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape # torch.Size([4, 8])

torch.Size([4, 8])

#### **解码器**

In [34]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, attention_size)
        # GRU的输入包含attention输出的c和实际输入，实际尺寸是2*embed_size
        self.rnn = nn.GRU(2*embed_size, num_hiddens, num_layers, dropout=drop_prob)
        self.out = nn.Linear(num_hiddens, vocab_size)
        
    def forward(self, cur_input, state, enc_states):
        """
        cur_input:(batch, )
        state:(num_layer, batch, num_hiddens)
        """
        # 取state的最后一层计算第一步的attention参数
        c = attention_forward(self.attention, enc_states, state[-1])
        # 将嵌入后的输入和背景向量相连接
        input_and_c = torch.cat((self.embedding(cur_input), c), dim=1) # (batch, 2*embed_size)
        # 为输入和背景向量增加一个时间步维度，时间步个数为1，就是一个时间步为1的GRU
        output, state = self.rnn(input_and_c.unsqueeze(0), state)
        # 移除时间步维度，输出形状(批量大小, 输出词典大小)
        output = self.out(output).squeeze(dim=0)
        return output, state
    
    def begin_state(self, enc_state):
        return enc_state

#### **损失计算过程**

我们先实现`batch_loss`函数计算一个小批量的损失。解码器在最初时间步的输入是特殊字符`BOS`。之后，解码器在某时间步的输入为样本输出序列在上一时间步的词，即强制教学。此外，同word2vec中的实现一样，我们在这里也使用掩码变量避免填充项对损失函数计算的影响。

In [35]:
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state() # 将encoder的初始状态设置为0
    # 计算encoder
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state  = decoder.begin_state(enc_state)
    # 解码器初始时间的输入是BOS
    dec_input = torch.tensor([out_vocab.stoi[BOS]] * batch_size)
    # 掩码变量需要用来忽略标签为填充项的损失
    mask, num_not_pad_tokens = torch.ones(batch_size,), 0
    l = torch.tensor([0.0])
    for y in Y.permute(1, 0): # Y(batch, seq_len)
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y # 使用强制教学，即真实的值作为数据到下一步时间步的输入
        num_not_pad_tokens += mask.sum().item() # 计算当前time_step的非pad元素个数并加入
        mask = mask * (y != out_vocab.stoi[PAD]).float() # 将PAD对应位置的掩码设置为0
    return l / num_not_pad_tokens

In [36]:
def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss(reduction='none')
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
    for epoch in range(num_epochs):
        l_sum = 0.0
        for X, Y in data_iter:
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            l = batch_loss(encoder, decoder, X, Y, loss)
            l.backward()
            enc_optimizer.step()
            dec_optimizer.step()
            l_sum += l.item()
        if (epoch + 1) % 10 == 0:
            print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter))) 

In [37]:
embed_size, num_hiddens, num_layers = 64, 64, 2
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers,
                  drop_prob)
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers,
                  attention_size, drop_prob)
train(encoder, decoder, dataset, lr, batch_size, num_epochs)

epoch 10, loss 0.403
epoch 20, loss 0.185
epoch 30, loss 0.087
epoch 40, loss 0.060
epoch 50, loss 0.039


#### **预测**

In [38]:
def translate(encoder, decoder, input_seq, max_seq_len):
    in_tokens = input_seq.split(' ')
    in_tokens += [EOS] + [PAD] * (max_seq_len - len(in_tokens) - 1)
    enc_input = torch.tensor([[in_vocab.stoi[tk] for tk in in_tokens]]) # batch=1
    enc_state = encoder.begin_state()
    enc_output, enc_state = encoder(enc_input, enc_state)
    dec_input = torch.tensor([out_vocab.stoi[BOS]])
    dec_state = decoder.begin_state(enc_state)
    output_tokens = []
    for _ in range(max_seq_len):
        dec_output, dec_state = decoder(dec_input, dec_state, enc_output)
        pred = dec_output.argmax(dim=1)
        pred_token = out_vocab.itos[int(pred.item())]
        if pred_token == EOS:  # 当任一时间步搜索出EOS时，输出序列即完成
            break
        else:
            output_tokens.append(pred_token)
            dec_input = pred
    return output_tokens

In [39]:
input_seq = 'ils regardent .'
translate(encoder, decoder, input_seq, max_seq_len)

['they', 'are', 'watching', '.']

我们通常使用BLEU来计算翻译的准确率

BLEU在nltk包有实现

In [41]:
from nltk.translate.bleu_score import sentence_bleu

In [45]:
ground_truth = ['I', 'love', 'apple', 'and', 'orange']
pred_sts = [['I', 'love', 'banana', 'and', 'orange'], ['I', 'love', 'apple', 'and', 'orange']]
sentence_bleu(pred_sts, ground_truth)

1.0